<a href="https://colab.research.google.com/github/pseudope/2021_F_Level2/blob/main/Projects/2%EC%A1%B0/3i4k_1008_%EC%9E%84%EC%8B%9C_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

import konlpy
from konlpy.tag import Okt
from keras.preprocessing.text import Tokenizer

from keras.layers import Embedding, Dense, LSTM, Bidirectional, Dropout
from keras.models import Sequential 
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

### 데이터 다운로드

---

In [3]:
!wget https://raw.githubusercontent.com/warnikchow/3i4k/master/data/train_val_test/fci_train_val.txt
!wget https://raw.githubusercontent.com/warnikchow/3i4k/master/data/train_val_test/fci_test.txt

--2021-10-07 18:52:32--  https://raw.githubusercontent.com/warnikchow/3i4k/master/data/train_val_test/fci_train_val.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2661060 (2.5M) [text/plain]
Saving to: ‘fci_train_val.txt’

fci_train_val.txt   100%[===================>]   2.54M  --.-KB/s    in 0.08s   

2021-10-07 18:52:34 (32.6 MB/s) - ‘fci_train_val.txt’ saved [2661060/2661060]

--2021-10-07 18:52:34--  https://raw.githubusercontent.com/warnikchow/3i4k/master/data/train_val_test/fci_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 

In [4]:
#데이터 불러오기

def read_data(filename):

    with open(filename, "r", encoding="UTF8") as f:
        data = [line.split("\t") for line in f.read().splitlines()]

    return data

In [5]:

train_data = read_data("/content/fci_train_val.txt")


#X,y
X_train_data = [t[1] for t in train_data]
y_train_data = [int(t[0]) for t in train_data]



test_data = read_data("/content/fci_test.txt")


#X,y
X_test_data = [t[1] for t in test_data]
y_test_data = [int(t[0]) for t in test_data]

In [6]:
print(len(X_train_data))
print(len(y_train_data))
print(len(X_test_data))
print(len(y_test_data))

55134
55134
6121
6121


In [7]:
#불용어 사전 가져오기

df = pd.read_csv('https://raw.githubusercontent.com/cranberryai/todak_todak_python/master/machine_learning_text/clean_korean_documents/korean_stopwords.txt', header=None)
df[0] = df[0].apply(lambda x: x.strip())

stopwords = df[0].to_numpy()

print(stopwords)
print(len(stopwords))

['이' '있' '하' '것' '들' '그' '되' '수' '이' '보' '않' '없' '나' '사람' '주' '아니' '등' '같'
 '우리' '때' '년' '가' '한' '지' '대하' '오' '말' '일' '그렇' '위하' '때문' '그것' '두' '말하'
 '알' '그러나' '받' '못하' '일' '그런' '또' '문제' '더' '사회' '많' '그리고' '좋' '크' '따르' '중'
 '나오' '가지' '씨' '시키' '만들' '지금' '생각하' '그러' '속' '하나' '집' '살' '모르' '적' '월' '데'
 '자신' '안' '어떤' '내' '경우' '명' '생각' '시간' '그녀' '다시' '이런' '앞' '보이' '번' '나' '다른'
 '어떻' '여자' '개' '들' '사실' '이렇' '점' '싶' '말' '정도' '좀' '원' '잘' '통하' '소리' '놓']
98


### 02. 토큰화 및 인코딩 (augmentation 생략)

In [8]:
!pip install konlpy
!pip install soynlp

In [9]:
okt = Okt()

#형태소 토큰화


X_train_new=[]
for sentence_new in X_train_data : 
    temp_X = [] 
    temp_X = okt.morphs(sentence_new, stem=True) # 토큰화 
    temp_X = [word for word in temp_X if not word in stopwords] #불용어에 해당하지 않는 토큰만 저장
    X_train_new.append(temp_X)

X_test_new = [] 
for sentence_new in X_test_data : 
    temp_X = [] 
    temp_X = okt.morphs(sentence_new, stem=True) # 토큰화 
    temp_X = [word for word in temp_X if not word in stopwords] #불용어에 해당하지 않는 토큰만 저장
    X_test_new.append(temp_X)

In [10]:
print(X_train_new[:5])
print(X_test_new[:5])

[['만화'], ['이치'], ['약'], ['사이'], ['짜다']]
[['연두색'], ['되다'], [], ['그걸루'], ['연구소']]


In [11]:
#정수 인코딩

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_new)

total_cnt = len(tokenizer.word_index)
vocab_size = total_cnt + 1

In [12]:
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train_new)
X_train = tokenizer.texts_to_sequences(X_train_new)
X_test = tokenizer.texts_to_sequences(X_test_new)

In [13]:
X_train[:5]

[[3906], [7779], [389], [568], [915]]

In [14]:
#15로 패딩

from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 15
# 한국어 특성상 문장 끝이 중요하므로 오른쪽에 치우치게 패딩

X_train_pad_new = pad_sequences(X_train, maxlen = max_len)
X_test_pad_new = pad_sequences(X_test, maxlen = max_len)

print(X_train_pad_new)
print(len(X_train_pad_new))
print(len(X_test_pad_new))

[[   0    0    0 ...    0    0 3906]
 [   0    0    0 ...    0    0 7779]
 [   0    0    0 ...    0    0  389]
 ...
 [   0    0    0 ...    0  317   32]
 [   0    0    0 ...    0   20   66]
 [   0    0    0 ... 1075  110    1]]
55134
6121


In [15]:
from tensorflow.keras.utils import to_categorical
print(len(set(y_train_data)))
y_train = to_categorical(y_train_data,len(set(y_train_data)))
y_test = to_categorical(y_test_data,len(set(y_train_data)))


print(len(X_train_pad_new))
print(len(X_test_pad_new))
print(len(y_train))
print(len(y_test))

7
55134
6121
55134
6121


In [16]:
from sklearn.model_selection import train_test_split

X_train_new, X_valid_new, y_train_new, y_valid_new = train_test_split(X_train_pad_new, y_train, test_size=0.1, shuffle=True, stratify=y_train)
X_test_new = X_test_pad_new
y_test_new = y_test
print(y_valid_new[0])
print(len(X_valid_new))
print(len(y_valid_new))

[0. 0. 1. 0. 0. 0. 0.]
5514
5514


In [17]:
from sklearn.model_selection import train_test_split

X_train_new, X_valid_new, y_train_new, y_valid_new = train_test_split(X_train_pad_new, y_train, test_size=0.1, shuffle=True, stratify=y_train)
X_test_new = X_test_pad_new
y_test_new = y_test
print(y_valid_new[0])
print(len(X_valid_new))
print(len(y_valid_new))

[0. 0. 1. 0. 0. 0. 0.]
5514
5514


## Modeling 1

In [18]:
!pip install tensorflow-addons

from tensorflow_addons.metrics import F1Score

     |████████████████████████████████| 1.1 MB 4.2 MB/s 


In [19]:
f1 = F1Score(num_classes=7, name="F1")
weighted_f1 = F1Score(num_classes=7, average="weighted", name="Weighted_F1")

In [20]:
#pretrained data 다운로드 후 사용 (코랩 용)
!pip install gensim
!pip install gdown

In [21]:
!gdown --id 1jHbjOcnaLourFzNuP47yGQVhBTq6Wgor --output vector.zip

Downloading...
From: https://drive.google.com/uc?id=1jHbjOcnaLourFzNuP47yGQVhBTq6Wgor
To: /content/vector.zip
100% 874M/874M [00:10<00:00, 79.6MB/s]


In [22]:
!unzip -qq "/content/vector.zip"

In [23]:
from gensim.models.fasttext import FastText as FT_gensim
loaded_model = FT_gensim.load_fasttext_format('/content/model_drama.bin')
embedding_matrix = np.zeros((vocab_size, 100))  
for word, i in tokenizer.word_index.items():
    if word in loaded_model:
        embedding_vector = loaded_model[word]
    else:
        continue     
    embedding_matrix[i] = embedding_vector

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [24]:
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from keras.models import Model

In [25]:
max_features = 16986
maxlen =15
embed_size = 100

In [26]:
def get_model(): #GRU 기반 모델, kaggle의 toxic classification 코드에서 따옴
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(GRU(80, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(7, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=["acc", f1, weighted_f1])

    return model

In [43]:
model=get_model()
history = model.fit(X_train_new, y_train_new, 
                    epochs=50, callbacks=[es, mc], batch_size=500, validation_data = (X_valid_new, y_valid_new))

Epoch 1/50
100/100 [==============================] - 7s 29ms/step - loss: 1.1473 - acc: 0.5912 - F1: 0.4080 - Weighted_F1: 0.5796 - val_loss: 0.7905 - val_acc: 0.7347 - val_F1: 0.4678 - val_Weighted_F1: 0.7043

Epoch 00001: val_acc improved from -inf to 0.73468, saving model to best_model.h5
Epoch 2/50
100/100 [==============================] - 2s 19ms/step - loss: 0.7266 - acc: 0.7579 - F1: 0.5356 - Weighted_F1: 0.7366 - val_loss: 0.6955 - val_acc: 0.7691 - val_F1: 0.5563 - val_Weighted_F1: 0.7507

Epoch 00002: val_acc improved from 0.73468 to 0.76913, saving model to best_model.h5
Epoch 3/50
100/100 [==============================] - 2s 19ms/step - loss: 0.6235 - acc: 0.7892 - F1: 0.6022 - Weighted_F1: 0.7744 - val_loss: 0.6422 - val_acc: 0.7740 - val_F1: 0.5660 - val_Weighted_F1: 0.7546

Epoch 00003: val_acc improved from 0.76913 to 0.77403, saving model to best_model.h5
Epoch 4/50
100/100 [==============================] - 2s 19ms/step - loss: 0.5617 - acc: 0.8081 - F1: 0.6528 - W

In [37]:
def get_model_two():
    model = Sequential()
    model.add(Embedding(max_features,100))
    model.add(LSTM(64))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(7, activation='softmax'))    
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=["acc", f1, weighted_f1])
    return model

In [42]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [39]:
model=get_model_two()
history = model.fit(X_train_new, y_train_new, 
                    epochs=50, callbacks=[es, mc], batch_size=500, validation_data = (X_valid_new, y_valid_new))

Epoch 1/50
100/100 [==============================] - 5s 24ms/step - loss: 1.1642 - acc: 0.5711 - F1: 0.3456 - Weighted_F1: 0.5323 - val_loss: 0.7481 - val_acc: 0.7448 - val_F1: 0.4568 - val_Weighted_F1: 0.7095

Epoch 00001: val_acc improved from -inf to 0.74483, saving model to best_model.h5
Epoch 2/50
100/100 [==============================] - 1s 13ms/step - loss: 0.7068 - acc: 0.7583 - F1: 0.4854 - Weighted_F1: 0.7275 - val_loss: 0.7507 - val_acc: 0.7450 - val_F1: 0.5049 - val_Weighted_F1: 0.7130

Epoch 00002: val_acc improved from 0.74483 to 0.74501, saving model to best_model.h5
Epoch 3/50
100/100 [==============================] - 1s 12ms/step - loss: 0.5888 - acc: 0.7985 - F1: 0.6143 - Weighted_F1: 0.7839 - val_loss: 0.6047 - val_acc: 0.7913 - val_F1: 0.6472 - val_Weighted_F1: 0.7826

Epoch 00003: val_acc improved from 0.74501 to 0.79126, saving model to best_model.h5
Epoch 4/50
100/100 [==============================] - 2s 16ms/step - loss: 0.5176 - acc: 0.8235 - F1: 0.7013 - W

In [44]:
loaded_model = load_model('best_model.h5')

In [45]:
loaded_model.evaluate(X_test_new, y_test_new)

192/192 [==============================] - 2s 6ms/step - loss: 0.5706 - acc: 0.8170 - F1: 0.7101 - Weighted_F1: 0.8103


[0.5705780982971191,
 0.8170233368873596,
 array([0.91848266, 0.811247  , 0.8777778 , 0.8086587 , 0.4047619 ,
        0.63095236, 0.51891893], dtype=float32),
 0.8102699518203735]